<a href="https://colab.research.google.com/github/clovesrocha/AnaliseDeSentimentos/blob/main/An%C3%A1lise_de_Sentimentos_Usando_TextBlob_NLTK%2BVader_e_Flair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Sentimento Simples
Neste documento faremos uma análise de sentimento simples usando 3 bibliotecas distintas para compararmos os resultados. Sendo elas:
- TextBlob
- NLTK + Vader
- Flair

In [ ]:
#lib TextBlob para análise de sentimento e tradução
from textblob import TextBlob

#lib nltk para análise de sentimento
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#lib flair para análise de sentimento
import flair
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


### No bloco a seguir está o texto que será analisado em ambos os modelos

In [ ]:
#preenchimento do texto a ser analisado (Em português)
text = 'Não é um bom aplicativo'

### As bibliotecas são pre-treinadas e funcionam melhor em inglês, então vamos traduzir o texto usando o TextBlob para utilizar nas predições

In [ ]:
#criando objeto TextBlob para tratar o texto em português e traduzir para inglês (idioma suportado pelas libs)
blob = TextBlob(text)
blob_t = blob.translate(from_lang='pt-br', to='en')
text_translated = str(blob_t)

#Análise usando o TextBlob
O Textblob possui uma função de análise de polaridade da sentença e através dela podemos definir se o texto tem sentimento positivo, negativo ou neutro. Neste caso usaremos o texto traduzido como sentença para avaliar seu sentimento.
Para isto, valores negativos de polaridade serão considerados como sentimento negativo, valores de polaridade positivos serão considerados como sentimentos positivos e polaridade igual a 0 serão sentimentos neutros.

In [ ]:
#fazendo análise de polaridade usando a lib TextBlob
p = blob_t.sentiment.polarity
if p < 0:
  result = 'Negativo'
elif p > 0:
  result = 'Positivo'
elif p == 0:
  result = 'Neutro'
else:
  result = 'Indefinido'

print(f'Resultado: Sentimento {result}')
print(f'Polaridade: {p}')

Resultado: Sentimento Negativo
Polaridade: -0.35


Acima podemos verificar que a polaridade do texto foi negativa, então o sentimento predito pela análise de sentimento do TextBlob é negativo.


#Análise usando NLTK + Vader
Abaixo usaremos a biblioteca NLTK com a análise de sentimento pre-treinada VADER. Esta função calcula valores positivos, negativos, neutros e o compound. Neste caso para definir se o sentimento é positivo, negativo ou neutro, usaremos o valor do compound para definir o resultado. Sendo assim, quando o compound for menor que -0,05 consideramos que o sentimento é negativo, quando o compound está entre -0,05 e 0,05, consideramos o sentimento como neutro e quando o compouns está acima de 0,05 consideramos o sentimento como positivo.

In [ ]:
#Fazendo análise de sentimento do texto usando a lib nltk e vader
sid = SentimentIntensityAnalyzer()
scores = sid.polarity_scores(text_translated)
compound = scores['compound']

if compound >= 0.05:
  result = 'Positivo'
elif compound > -0.05 and compound < 0.05:
  result = 'Neutro'
elif compound <= -0.05:
  result = 'Negativo'
else:
  result = 'Indefinido'


print(f'Resultado: Sentimento {result}')
print(f'Compound: {compound}')
print(f'Scores: {scores}')

Resultado: Sentimento Negativo
Compound: -0.3412
Scores: {'neg': 0.376, 'neu': 0.624, 'pos': 0.0, 'compound': -0.3412}


Acima podemos ver que o compound foi negativo e portando o sentimento é negativo.


#Análise de Sentimento com Flair
A predição de sentimento feita pela biblioteca flair é especificamente com o retorno positivo ou negativo da senteça, com uma margem de confiança.
Quando o valor do retorno é vazio, então a sentença pode ser considerada neutra.

In [ ]:
#fazendo analise de sentimento usando a lib flair
s = flair.data.Sentence(text_translated)
flair_sentiment.predict(s)
total_sentiment = s.labels
total_sentiment

s.to_dict()
s.to_dict()['text']
s.to_dict()['all labels'][0]['value']
s.to_dict()['all labels'][0]['confidence']

print(f"Texto avaliado: {s.to_dict()['text']}")
print(f"Sentimento: {s.to_dict()['all labels'][0]['value']}")
print(f"Confidence: {s.to_dict()['all labels'][0]['confidence']}")


Texto avaliado: It is not a good application
Sentimento: NEGATIVE
Confidence: 0.9996412992477417


Acima podemos ver que a sentença foi considerada negativa com uma margem de confiança de 99%

#Conclusão
Todos os modelos indicaram o sentimento correto da sentença, cabe ao desenvolvedor analisar mais a fundo cada um e testar em uma base de dados maior onde possa avaliar a performance.

Próximos passos:
- Testar novas senteças para definir qual modelo funciona melhor;
- Testar com uma base de dados com muitas sentenças e que já estejam classificadas corretamente, para assim, conseguir comparar e avaliar o desempenho do modelo;
- Verificar formas de melhorar a performance, personalizando ou ajustando hiperparametros;
- Colocar em produção criando um batch job que classifica uma base de dados ou gerando uma api que classifica através de um endpoint que receba as sentenças;